# Ising Model using Monte Carlo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants

In [2]:
class Ising:
    def __init__(self,x,y,B=1.0,J=1.0,M=1.0,kT=1.0,xr=2,yr=2):
        self.xlen = x
        self.ylen = y
        self.config = np.ones((x,y),dtype=int)
        self.mag = B
        self.coup = J
        self.mu = M
        self.kT = kT
        self.xr = xr
        self.yr = yr
        self.prob = 0
        
    def __str__(self):
        return str(self.config)
    
    def reset(self):
        self.config = np.ones((self.xlen,self.ylen),dtype=int)
        
    def random(self):
        for i in range(self.xlen):
            for j in range(self.ylen):
                ck = np.random.randint(2)
                if ck != 0 :
                    self.config[i][j] = 1
                else:
                    self.config[i][j] = -1
        
    def magnetization(self):
        return self.mu*np.sum(self.config)/(self.xlen*self.ylen)
    
    def energy(self):
        V = -1 * self.mag * np.sum(self.config)
        V *= self.xr + self.yr - 1
        
        temp = 0
        temp1=0
        temp2=0
        for i in range(self.xlen):
            for j in range(self.ylen):
                if i != self.xlen - 1:
                    temp -= self.config[i][j] * self.config[i+1][j]
                if j != self.ylen - 1:
                    temp -= self.config[i][j] * self.config[i][j+1]
        temp *= (self.xr + self.yr - 1)
        
        for j in range(self.ylen):
            temp1 -= self.config[-1][j] * self.config[0][j]
        temp1 *= self.xr
        
        for j in range(self.xlen):
            temp2 -= self.config[j][-1] * self.config[j][0]
        temp2 *= self.yr
        
        temp += temp1 + temp2
        temp*=self.coup
        
        V += temp
        return V
        
        
    def MonteCarlo(self):
        nx = np.random.randint(0,self.xlen)
        SumSpin=0
        ny = np.random.randint(0,self.ylen)
        if nx!=self.xlen-1:
            SumSpin+=self.config[nx+1][ny]
        else:
            SumSpin+=self.config[0][ny]
        if ny!=self.ylen-1:
            SumSpin+=self.config[nx][ny+1]
        else:
            SumSpin+=self.config[nx][0]
        if nx!=0:
            SumSpin+=self.config[nx-1][ny]
        else:
            SumSpin+=self.config[-1][ny]
        if ny!=0:
            SumSpin+=self.config[nx][ny-1]
        else:
            SumSpin+=self.config[nx][-1]        
        
        V = self.config[nx,ny]*(self.mag*self.mu*2 + self.coup*SumSpin)
        V *= self.xr + self.yr -1
        p_acc = np.exp(-1.0/self.kT*V)
        if V < 0 or np.random.random() < p_acc:
            self.config[nx,ny]*=-1
            accept=True
        else:
            accept=False
        self.prob = p_acc
        return accept
    
    def Mag_Step(self,stepSize,totStep):
        Step=0
        E=0
        U=0
        while Step < totStep:
            flag = self.MonteCarlo()
            Step+=1
            if Step%stepSize != 0:
                continue
            else:
                E = self.magnetization()
        E/=Step
        return E   

In [3]:
ising = Ising(50,50,0)
ising.mag = 0 
res = ising.energy()

In [ ]:
ising.kT = 1
ising.random()
x = []
for i in range(20000):
    flag = ising.MonteCarlo()
    en = ising.energy()
    x.append(en)

In [ ]:
plt.plot(x)
plt.title('Energy vs Temperature')
plt.xlabel('T')
plt.ylabel('Energy')
plt.show()

Energy vs Temperature

In [ ]:
ising.mag = 0
s = 100
ss = 100
ttf = 100
mmt = 0.5
x = []
y = []
for i in np.linspace(mmt,ttf,300):
    ising.kT = i
    mg = ising.Mag_Step(s,ss)
    x.append(mg)
    y.append(i)

 Magnetization vs Temperature

In [ ]:
plt.plot(y,x)
plt.title('Magnetization vs Temperature')
plt.xlabel('kbT')
plt.ylabel('Magnetization')
plt.show()